# Proyecto Final: Análisis de la Encuesta de Desarrolladores de Stack Overflow 2025 👨‍💻

## Alumno
- Fernando Javier García Liévano

## Abstract

Durante mi proyecto, estaré llevando a cabo un análisis exploratorio enfocado en un conjunto de datos obtenido de la **Encuesta a Desarrolladores de Stack Overflow 2025**. Esta encuesta es ampliamente reconocida en la comunidad tecnológica, ya que recopila información de miles de desarrolladores alrededor del mundo, brindándonos una visión amplia del estado actual de la industria del desarrollo de software. Los datos en el conjunto seleccionado incluyen aspectos como los lenguajes de programación más utilizados, tecnologías en tendencia, roles más comunes dentro de las empresas, niveles salariales y otros datos relevantes para comprender mejor el ecosistema del desarrollo de software.

El objetivo principal de esta fase del proyecto es **explorar y entender el dataset a fondo**. Para eso, voy a usar técnicas de limpieza y transformación de datos, análisis estadístico y visualizaciones que me permitan encontrar patrones, tendencias y relaciones interesantes. Quiero responder a preguntas como: **¿qué lenguajes de programación están en auge?**, **¿qué tecnologías están perdiendo relevancia?** o **¿cómo varían los salarios dependiendo de la región, el rol o los años de experiencia?**.

Este análisis exploratorio también me servirá como punto de partida para la siguiente fase del proyecto, donde realizaremos la selección de un modelo de Aprendizaje Automático para una futura implementación sobre el conjunto de datos seleccionado.

En resumen, durante este proyecto espero practicar y fortalecer mis habilidades en **Ciencia de Datos**, Python y otras herramientas clave en el manejo de datos, además de entender mejor la realidad de la industria de la que formo parte y así fortalecer mi perfil profesional para crecer y desarrollarme dentro del mercado laboral.


## Preguntas e Hipótesis de Interés
1. ¿Qué lenguajes de programación están en auge?
2. ¿Qué tecnologías están perdiendo relevancia?
3. ¿Cómo varían los salarios dependiendo de la región, el rol o los años de experiencia?